In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
# import torch.nn.functional as F
from torch import optim
import sklearn

In [5]:
# read files
raw_data_initial = pd.read_csv('vital_triage.csv')
raw_data_sequential = pd.read_csv('ed_UPGI_vitalsigns_include_map.csv')
raw_data_lab = pd.read_csv('lab.csv')
raw_data_static = pd.read_csv('ugib-2.csv')

In [6]:
# build dataframe
data_initial = pd.DataFrame(raw_data_initial)
data_sequential = pd.DataFrame(raw_data_sequential)
data_lab = pd.DataFrame(raw_data_lab)
data_static = pd.DataFrame(raw_data_static)
data_dynamic = pd.concat([data_initial,data_sequential])
# combine the initial ED data and sequential data
data_dynamic
data_sequential

subject_id   stay_id       charttime  heartrate    sbp   dbp  \
0        10003019  30480460    6/1/76 12:41      100.0  126.0  75.0   
1        10003019  30480460    6/1/76 15:38       97.0  133.0  70.0   
2        10003019  30480460    6/1/76 16:10       97.0  132.0  65.0   
3        10003299  33780565  10/12/78 18:19       58.0  149.0  71.0   
4        10003299  33780565  10/12/78 19:28       68.0  121.0  65.0   
...           ...       ...             ...        ...    ...   ...   
15275    19992875  38038768   27/9/61 23:57       63.0  129.0  79.0   
15276    19992875  38038768    28/9/61 0:50       66.0  118.0  77.0   
15277    19992875  38038768    28/9/61 3:40       62.0   96.0  59.0   
15278    19997367  38942696   13/3/26 15:43       81.0  112.0  58.0   
15279    19997367  38942696   13/3/26 18:28       87.0  108.0  72.0   

               map  transfusion  
0               92          NaN  
1               91          NaN  
2      87.33333333          NaN  
3               97          NaN  
4      83.66666667          NaN  
...            ...          ...  
15275  95.66666667          NaN  
15276  90.66666667          NaN  
15277  71.33333333          NaN  
15278           76          NaN  
15279           84          NaN  

[15280 rows x 8 columns]

In [7]:
# screening the data by MAP > 60 & Hb > 7 & 6 hour
data_dynamic.map = data_dynamic.sbp * 1/3 + data_dynamic.dbp * 2/3


In [12]:
# add static data and labels
data_dynamic.merge(data_static, on='hadm_id', how='left')

subject_id_x     hadm_id  stay_id_x            charttime  heartrate  \
0          10003019  20962108.0   30480460  2176-01-06 11:14:00      113.0   
1          10003299  28891311.0   33780565  2178-12-10 17:04:00       58.0   
2          10003400  29483621.0   34948767  2136-11-04 16:08:00       86.0   
3          10005024  25023471.0   30375901  2138-03-28 23:30:00      100.0   
4          10006368  28366563.0   36293519  2178-08-25 12:07:00       84.0   
...             ...         ...        ...                  ...        ...   
17603      19992875         NaN   38038768        27/9/61 23:57       63.0   
17604      19992875         NaN   38038768         28/9/61 0:50       66.0   
17605      19992875         NaN   38038768         28/9/61 3:40       62.0   
17606      19997367         NaN   38942696        13/3/26 15:43       81.0   
17607      19997367         NaN   38942696        13/3/26 18:28       87.0   

         sbp    dbp         map  transfusion  subject_id_y   stay_id_y  \
0      112.0   75.0   87.333333          NaN    10003019.0  30480460.0   
1      155.0   80.0  105.000000          NaN    10003299.0  33780565.0   
2      142.0   92.0  108.666667          NaN    10003400.0  34948767.0   
3      164.0  116.0  132.000000          NaN    10005024.0  30375901.0   
4      114.0   72.0   86.000000          NaN    10006368.0  36293519.0   
...      ...    ...         ...          ...           ...         ...   
17603  129.0   79.0   95.666667          NaN           NaN         NaN   
17604  118.0   77.0   90.666667          NaN           NaN         NaN   
17605   96.0   59.0   71.333333          NaN           NaN         NaN   
17606  112.0   58.0   76.000000          NaN           NaN         NaN   
17607  108.0   72.0   84.000000          NaN           NaN         NaN   

                    intime              outtime gender  \
0      2176-01-06 11:14:00  2176-01-06 16:55:00      M   
1      2178-12-10 17:04:00  2178-12-10 21:36:00      F   
2      2136-11-04 16:08:00  2136-11-04 22:12:00      F   
3      2138-03-28 23:30:00  2138-03-29 03:08:00      M   
4      2178-08-25 12:07:00  2178-08-25 17:04:54      F   
...                    ...                  ...    ...   
17603                  NaN                  NaN    NaN   
17604                  NaN                  NaN    NaN   
17605                  NaN                  NaN    NaN   
17606                  NaN                  NaN    NaN   
17607                  NaN                  NaN    NaN   

                         race        age trans_time  trans  
0                       WHITE  74.014925        NaN    0.0  
1      BLACK/AFRICAN AMERICAN  70.940787        NaN    0.0  
2      BLACK/AFRICAN AMERICAN  74.844315        NaN    0.0  
3                     UNKNOWN  60.238345        NaN    0.0  
4             WHITE - RUSSIAN  72.648086        NaN    0.0  
...                       ...        ...        ...    ...  
17603                     NaN        NaN        NaN    NaN  
17604                     NaN        NaN        NaN    NaN  
17605                     NaN        NaN        NaN    NaN  
17606                     NaN        NaN        NaN    NaN  
17607                     NaN        NaN        NaN    NaN  

[17608 rows x 18 columns]